## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]
y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
feature_cols = ["HourlyRate","JobInvolvement", "JobLevel", "JobRole", "JobSatisfaction", "OverTime", "PercentSalaryHike", "PerformanceRating", "StockOptionLevel", "TotalWorkingYears"]


# Create X_df using your selected columns
X_df = attrition_df[feature_cols]

# Show the data types for X_df
display(X_df.head())
display(X_df.dtypes)

,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,OverTime,PercentSalaryHike,PerformanceRating,StockOptionLevel,TotalWorkingYears
0,94,3,2,Sales Executive,4,Yes,11,3,0,8
1,61,2,2,Research Scientist,2,No,23,4,1,10
2,92,2,1,Laboratory Technician,3,Yes,15,3,0,7
3,56,3,1,Research Scientist,3,Yes,11,3,0,8
4,40,3,1,Laboratory Technician,2,No,12,3,1,6


HourlyRate            int64
JobInvolvement        int64
JobLevel              int64
JobRole              object
JobSatisfaction       int64
OverTime             object
PercentSalaryHike     int64
PerformanceRating     int64
StockOptionLevel      int64
TotalWorkingYears     int64
dtype: object

## Note to grader: 
I was having trouble encoding after splitting the datasets, so I chose to encode before splitting via train_test_split. This process is similar to Module 18, day 2, assignment 6.

### Encode features dataframe:

In [5]:
# Generate our categorical variable lists
X_df_cat = X_df.dtypes[X_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
X_df[X_df_cat].nunique()

JobRole     9
OverTime    2
dtype: int64

In [6]:
# Create a OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
X_encode_df = pd.DataFrame(enc.fit_transform(X_df[X_df_cat]))

# Add the encoded variable names to the dataframe
X_encode_df.columns = enc.get_feature_names_out(X_df_cat)
X_encode_df.head()

,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,OverTime_No,OverTime_Yes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# Merge one-hot encoded features and drop the originals
X_df = X_df.merge(X_encode_df,left_index=True, right_index=True)
X_df = X_df.drop(X_df_cat, axis=1)
X_df.head()

,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,PercentSalaryHike,PerformanceRating,StockOptionLevel,TotalWorkingYears,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,OverTime_No,OverTime_Yes
0,94,3,2,4,11,3,0,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,61,2,2,2,23,4,1,10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,92,2,1,3,15,3,0,7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,56,3,1,3,11,3,0,8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,40,3,1,2,12,3,1,6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Encode target dataframe:

In [8]:
# Generate our categorical variable lists
y_df_cat = y_df.dtypes[y_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
y_df[y_df_cat].nunique()

Attrition     2
Department    3
dtype: int64

In [9]:
# Create a OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
encTarget = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
y_encode_df = pd.DataFrame(encTarget.fit_transform(y_df[y_df_cat]))

# Add the encoded variable names to the dataframe
y_encode_df.columns = encTarget.get_feature_names_out(y_df_cat)
y_encode_df.head()

,Attrition_No,Attrition_Yes,Department_Human Resources,Department_Research & Development,Department_Sales
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0
3,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
y_df = y_df.merge(y_encode_df,left_index=True, right_index=True)
y_df = y_df.drop(y_df_cat, axis=1)

In [13]:
# separate the 2 y variables
y_df_attrition = y_df['Attrition_Yes']
y_df_department = y_df[['Department_Human Resources','Department_Research & Development','Department_Sales']]

display(y_df_attrition.head())
display(y_df_department.head())

0    1.0
1    0.0
2    1.0
3    0.0
4    0.0
Name: Attrition_Yes, dtype: float64

,Department_Human Resources,Department_Research & Development,Department_Sales
0,0.0,0.0,1.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0


### Split into testing and training datasets:

In [14]:
# Ensure targets are NumPy arrays with correct shapes
y_df_attrition = y_df_attrition.to_numpy()
y_df_attrition = y_df_attrition.reshape(-1,1)
y_df_department = y_df_department.values

display(y_df_attrition.shape)  # Shape: (num_samples, 1)
display(y_df_department.shape) # Shape: (num_samples, 3)

(1470, 1)

(1470, 3)

In [15]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_attrition_train, y_attrition_test, y_department_train, y_department_test = train_test_split(
    X_df, y_df_attrition, y_df_department, test_size=0.2, random_state=42
)

## Create, Compile, and Train the Model

In [16]:
# Find the number of columns in the X training data
X_training_cols = len(X_train.columns)

# Create the input layer
input_layer = layers.Input(shape=(X_training_cols,), name='input_features')

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [17]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_branch = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_branch)


In [26]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(1, activation='sigmoid', name='attrition_output')(attrition_branch)


In [27]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss=['categorical_crossentropy', 'binary_crossentropy'],
    metrics=['accuracy', 'accuracy']
)


# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_features      │ (None, 19)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      1,280 │ input_features[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 32)        │      2,080 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 32)        │      1,056 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │      1,056 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 1)         │         33 │ dense_5[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,604 (21.89 KB)

 Trainable params: 5,604 (21.89 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
display(X_train.shape)
display(y_department_train.shape)
display(y_attrition_train.shape)
display(y_department_test.shape)
display(y_attrition_test.shape)

(1176, 19)

(1176, 3)

(1176, 1)

(294, 3)

(294, 1)

In [29]:
# Train the model
model.fit(
    X_train,
    [y_department_train, y_attrition_train],
    epochs=50,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.8273 - attrition_output_loss: 0.3934 - department_output_accuracy: 0.9367 - department_output_loss: 0.2063 - loss: 0.5999 - val_attrition_output_accuracy: 0.8008 - val_attrition_output_loss: 0.4362 - val_department_output_accuracy: 0.9025 - val_department_output_loss: 0.2157 - val_loss: 0.6801
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8671 - attrition_output_loss: 0.3396 - department_output_accuracy: 0.9332 - department_output_loss: 0.1966 - loss: 0.5364 - val_attrition_output_accuracy: 0.7924 - val_attrition_output_loss: 0.4327 - val_department_output_accuracy: 0.9068 - val_department_output_loss: 0.1991 - val_loss: 0.6546
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8560 - attrition_output_loss: 0.3531 - department_output_accuracy: 0.9400 - department_output_loss: 0.1736 - loss: 0.5264 - val_attrition_output_accuracy: 0.8051 - val

In [30]:
# Evaluate the model with the testing data
test_results = model.evaluate(
    X_test,
    [y_department_test, y_attrition_test]
)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8492 - attrition_output_loss: 0.3872 - department_output_accuracy: 0.9467 - department_output_loss: 0.1536 - loss: 0.5475 


In [31]:
# Print the accuracy for both department and attrition
department_accuracy = test_results[3]  
attrition_accuracy = test_results[4] 


print(f"Department Accuracy: {department_accuracy:.4f}")
print(f"Attrition Accuracy: {attrition_accuracy:.4f}")

Department Accuracy: 0.8707
Attrition Accuracy: 0.9388


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. While my accuracy scores are relatively high, accuracy may be misleading if attrition data is imbalanced. Employee attrition scenarios are difficult to predict with accuracy because of imbalanced data. Other metrics may be more appropriate (e.g. F1-score, precision, or recall)

2. I chose 2 different activation functions:
    - Department: "softmax" was chosen because it is appropriate for multi-class classification
    - Attrition: "sigmoid" was chosen for binary classification
 
3. I can improve my model with the following strategies: 
    - Include more features from the original dataframe into the model
    - Include hyperparameter tuning, with techniques such as grid search or random search
    - Address any class imbalances with techniques such as oversampling and undersampling
    - Experiment with adding more layers or nuerons in the network 
